# Pivoting and Summarizing Student Grades


## Loading the Required Libraries

In [2]:
# Load Libraries
import polars as pl
import polars.selectors as cs
import sys 

# Display system and polars versions
print(f'My system version is {sys.version};\npolars version is {pl.__version__}')

My system version is 3.12.4 (main, Jul  1 2024, 00:48:18) [Clang 15.0.0 (clang-1500.3.9.4)];
polars version is 1.5.0
My system version is 3.12.4 (main, Jul  1 2024, 00:48:18) [Clang 15.0.0 (clang-1500.3.9.4)];
polars version is 1.5.0


## Creating a sample dataset

In [3]:
# Create a sample student grades dataset
import numpy as np
np.random.seed(254)
student_grades = {
    'name': ['Ayen', 'Akuien', 'Garang', 'Dut', 'Atong', 'Nyakang', 'Lokule', 'Gatjang', 'Ojuok', 'Omot'],
    'english': np.random.normal(loc=85, scale=12, size=10),
    'mathematics': np.random.normal(loc=80, scale=18, size=10),
    'data_science': np.random.normal(loc=75, scale=15, size=10)
}

In [4]:
student_grades_2 = (
    pl.DataFrame(student_grades)
    .with_columns(
        english=pl.when(pl.col('english') > 100).then(100).otherwise(pl.col('english').round(1)),
        mathematics=pl.when(pl.col('mathematics') > 100).then(100).otherwise(pl.col('mathematics').round(1)),
        data_science=pl.when(pl.col('data_science') > 100).then(100).otherwise(pl.col('data_science').round(1))
    )
)

# Inspect output
print(student_grades_2)

shape: (10, 4)
┌─────────┬─────────┬─────────────┬──────────────┐
│ name    ┆ english ┆ mathematics ┆ data_science │
│ ---     ┆ ---     ┆ ---         ┆ ---          │
│ str     ┆ f64     ┆ f64         ┆ f64          │
╞═════════╪═════════╪═════════════╪══════════════╡
│ Ayen    ┆ 100.0   ┆ 69.9        ┆ 88.2         │
│ Akuien  ┆ 87.3    ┆ 96.1        ┆ 59.4         │
│ Garang  ┆ 84.1    ┆ 83.8        ┆ 66.7         │
│ Dut     ┆ 70.2    ┆ 51.9        ┆ 74.2         │
│ Atong   ┆ 100.0   ┆ 100.0       ┆ 90.3         │
│ Nyakang ┆ 75.4    ┆ 99.7        ┆ 68.4         │
│ Lokule  ┆ 75.9    ┆ 94.9        ┆ 94.9         │
│ Gatjang ┆ 92.9    ┆ 78.4        ┆ 77.4         │
│ Ojuok   ┆ 93.1    ┆ 100.0       ┆ 66.0         │
│ Omot    ┆ 79.4    ┆ 78.6        ┆ 73.1         │
└─────────┴─────────┴─────────────┴──────────────┘
shape: (10, 4)
┌─────────┬─────────┬─────────────┬──────────────┐
│ name    ┆ english ┆ mathematics ┆ data_science │
│ ---     ┆ ---     ┆ ---         ┆ ---          │
│

In [5]:
# Compute each student average grade
avg_grade = (
    student_grades_2
    .with_columns(avg_grade=pl.mean_horizontal(cs.float()).round(1))
)

# inspect output
print(avg_grade)

shape: (10, 5)
┌─────────┬─────────┬─────────────┬──────────────┬───────────┐
│ name    ┆ english ┆ mathematics ┆ data_science ┆ avg_grade │
│ ---     ┆ ---     ┆ ---         ┆ ---          ┆ ---       │
│ str     ┆ f64     ┆ f64         ┆ f64          ┆ f64       │
╞═════════╪═════════╪═════════════╪══════════════╪═══════════╡
│ Ayen    ┆ 100.0   ┆ 69.9        ┆ 88.2         ┆ 86.0      │
│ Akuien  ┆ 87.3    ┆ 96.1        ┆ 59.4         ┆ 80.9      │
│ Garang  ┆ 84.1    ┆ 83.8        ┆ 66.7         ┆ 78.2      │
│ Dut     ┆ 70.2    ┆ 51.9        ┆ 74.2         ┆ 65.4      │
│ Atong   ┆ 100.0   ┆ 100.0       ┆ 90.3         ┆ 96.8      │
│ Nyakang ┆ 75.4    ┆ 99.7        ┆ 68.4         ┆ 81.2      │
│ Lokule  ┆ 75.9    ┆ 94.9        ┆ 94.9         ┆ 88.6      │
│ Gatjang ┆ 92.9    ┆ 78.4        ┆ 77.4         ┆ 82.9      │
│ Ojuok   ┆ 93.1    ┆ 100.0       ┆ 66.0         ┆ 86.4      │
│ Omot    ┆ 79.4    ┆ 78.6        ┆ 73.1         ┆ 77.0      │
└─────────┴─────────┴─────────────┴─────

In [6]:
# Extract two best grades per student
top_two_grades = (
    student_grades_2
    .unpivot(index='name', value_name='grade', variable_name='subject')
    .group_by('name')
    .agg(pl.all().top_k_by('grade', 2))
    .explode(pl.all().exclude('name'))
)

# Inspect output
print(top_two_grades)

shape: (20, 3)
┌─────────┬─────────────┬───────┐
│ name    ┆ subject     ┆ grade │
│ ---     ┆ ---         ┆ ---   │
│ str     ┆ str         ┆ f64   │
╞═════════╪═════════════╪═══════╡
│ Ojuok   ┆ mathematics ┆ 100.0 │
│ Ojuok   ┆ english     ┆ 93.1  │
│ Atong   ┆ english     ┆ 100.0 │
│ Atong   ┆ mathematics ┆ 100.0 │
│ Omot    ┆ english     ┆ 79.4  │
│ …       ┆ …           ┆ …     │
│ Gatjang ┆ mathematics ┆ 78.4  │
│ Garang  ┆ english     ┆ 84.1  │
│ Garang  ┆ mathematics ┆ 83.8  │
│ Akuien  ┆ mathematics ┆ 96.1  │
│ Akuien  ┆ english     ┆ 87.3  │
└─────────┴─────────────┴───────┘
shape: (20, 3)
┌─────────┬─────────────┬───────┐
│ name    ┆ subject     ┆ grade │
│ ---     ┆ ---         ┆ ---   │
│ str     ┆ str         ┆ f64   │
╞═════════╪═════════════╪═══════╡
│ Ojuok   ┆ mathematics ┆ 100.0 │
│ Ojuok   ┆ english     ┆ 93.1  │
│ Atong   ┆ english     ┆ 100.0 │
│ Atong   ┆ mathematics ┆ 100.0 │
│ Omot    ┆ english     ┆ 79.4  │
│ …       ┆ …           ┆ …     │
│ Gatjang ┆ mathem